In [55]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades

folder_path = r'C:\Users\bayli\Documents\CS Demos\All_IEM_Katowice_2025\IEM_Katowice_2025'
file_path = r'C:\Users\bayli\Documents\Git Projects\test_demos\natus-vincere-vs-faze-m1-inferno.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
i = 1

In [56]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()
    
    # Changes the first round losing steak to 0
    # This is necessary because the first round is not a losing streak
    ticks_df.loc[ticks_df['round_num'] == 1, ['ct_losing_streak', 't_losing_streak']] = 0

    # Makes sure the columns exists
    rounds_df['CT_team_clan_name'] = None
    rounds_df['T_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['CT_team_current_equip_value'] = None
    rounds_df['T_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Takes the losing streak for every team
        try:
            ct_losing_streak = first_tick_df[first_tick_df['side'] == 'ct']['ct_losing_streak'].iloc[0]
        except IndexError:
            ct_losing_streak = None

        try:
            t_losing_streak = first_tick_df[first_tick_df['side'] == 't']['t_losing_streak'].iloc[0]
        except IndexError:
            t_losing_streak = None

        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'CT_team_clan_name'] = CT_team
        rounds_df.at[idx, 'T_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'CT_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 'T_team_current_equip_value'] = T_team_current_equip_value
        rounds_df.at[idx, 'ct_losing_streak'] = ct_losing_streak
        rounds_df.at[idx, 't_losing_streak'] = t_losing_streak


    return rounds_df

In [ ]:
def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['CT_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['CT_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['CT_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['CT_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['T_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['T_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['T_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['T_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

In [58]:
def calculate_5v4_advantage(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['5v4_advantage'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']  # Exemplo: 'ct' ou 't'

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, '5v4_advantage'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, '5v4_advantage'] = 't'

    return rounds_df

In [59]:
dem = Demo(file_path)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak", "is_terrorist_timeout", "is_ct_timeout", "4k_rounds_total"])

this_file_df_ticks = dem.ticks
this_file_df_rounds = dem.rounds
this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)

In [60]:
this_file_df_rounds[['CT_buy_type', 'T_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')

In [61]:
# Opening Kills Data
this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
this_file_df_kills = this_file_df_kills.with_columns(
    this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
    this_file_df_kills['assister_steamid'].cast(pl.Utf8),
    this_file_df_kills['victim_steamid'].cast(pl.Utf8)
)
this_file_df_kills = this_file_df_kills.to_pandas()
first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
first_kills = first_kills.groupby('round_num').first().reset_index()
df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)

In [ ]:
this_file_df_rounds = calculate_5v4_advantage(this_file_df_rounds, df_all_first_kills)

In [63]:
this_file_df_rounds

,round_num,start,freeze_end,end,official_end,winner,reason,bomb_plant,bomb_site,CT_team_clan_name,T_team_clan_name,winner_clan_name,CT_team_current_equip_value,T_team_current_equip_value,ct_losing_streak,t_losing_streak,CT_buy_type,T_buy_type,5v4_advantage
0,1,540,6363,12448,12896,t,bomb_exploded,9824.0,bombsite_b,Natus Vincere,FaZe Clan,FaZe Clan,4200,4250,0,0,Pistol,Pistol,t
1,2,12896,14176,23643,24091,t,bomb_exploded,21019.0,bombsite_b,Natus Vincere,FaZe Clan,FaZe Clan,11950,20150,2,0,Semi Buy,Full Buy,t
2,3,24091,25371,29749,30197,ct,t_killed,NaN,not_planted,Natus Vincere,FaZe Clan,Natus Vincere,8350,22050,3,0,Semi Buy,Full Buy,ct
3,4,30197,31477,38837,39285,ct,time_ran_out,NaN,not_planted,Natus Vincere,FaZe Clan,Natus Vincere,25150,19400,1,1,Full Buy,Full Buy,ct
4,5,39285,40565,47343,47791,ct,t_killed,NaN,not_planted,Natus Vincere,FaZe Clan,Natus Vincere,28600,5200,0,2,Full Buy,Eco,ct
5,6,47791,49071,58805,59253,t,bomb_exploded,56181.0,bombsite_b,Natus Vincere,FaZe Clan,FaZe Clan,27800,20800,0,3,Full Buy,Full Buy,t
6,7,59253,63458,70056,70504,ct,bomb_defused,67585.0,bombsite_b,Natus Vincere,FaZe Clan,Natus Vincere,29650,25700,1,1,Full Buy,Full Buy,ct
7,8,70504,71784,78962,79410,t,ct_killed,77678.0,bombsite_b,Natus Vincere,FaZe Clan,FaZe Clan,29400,8950,0,2,Full Buy,Semi Buy,ct
8,9,79410,80690,88050,88498,ct,time_ran_out,NaN,not_planted,Natus Vincere,FaZe Clan,Natus Vincere,26050,28050,1,0,Full Buy,Full Buy,ct
9,10,88498,92116,95721,96169,ct,t_killed,NaN,not_planted,Natus Vincere,FaZe Clan,Natus Vincere,28550,10700,0,1,Full Buy,Semi Buy,ct
